In [1]:
import sys
import os

# Ruta al directorio raíz del proyecto (ajústala si es necesario)
ruta_raiz = os.path.abspath(r"C:\Users\Germán Llorente\Desktop\germiprogramer\TFG-Agente-de-Scouting")

if ruta_raiz not in sys.path:
    sys.path.append(ruta_raiz)

In [2]:
import importlib
import funciones_analisis.estadisticas

importlib.reload(funciones_analisis.estadisticas)
importlib.reload(funciones_analisis.generales)

from funciones_analisis.estadisticas import *
from funciones_analisis.generales import *
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch, Pitch
from highlight_text import ax_text, fig_text
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as path_effects
import seaborn as sns
import glob

In [3]:
ratings = pd.read_csv("datos/ratings/data_football_ratings.csv")

In [4]:
ratings = ratings[ratings['competition'] == 'World Cup 2018']

In [5]:
free_comps = sb.competitions()

mundial_2018 = sb.matches(competition_id=43, season_id=3)

In [6]:
mundial_2018

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,7585,2018-07-03,20:00:00.000,International - FIFA World Cup,2018,Colombia,England,1,1,available,...,2021-06-13T16:17:31.694,4,Round of 16,Otkritie Bank Arena,Mark Geiger,José Néstor Pekerman,Gareth Southgate,1.0.2,None,None
1,7570,2018-06-28,20:00:00.000,International - FIFA World Cup,2018,England,Belgium,0,1,available,...,2021-06-13T16:17:31.694,3,Group Stage,Stadion Kaliningrad,Damir Skomina,Gareth Southgate,Roberto Martínez Montoliú,1.0.2,None,None
2,7586,2018-07-03,16:00:00.000,International - FIFA World Cup,2018,Sweden,Switzerland,1,0,available,...,2021-06-13T16:17:31.694,4,Round of 16,Saint-Petersburg Stadium,Damir Skomina,Jan Olof Andersson,Vladimir Petković,1.0.2,None,None
3,7557,2018-06-25,20:00:00.000,International - FIFA World Cup,2018,Iran,Portugal,1,1,available,...,2021-06-13T16:17:31.694,3,Group Stage,Mordovia Arena,Enrique Cáceres,Carlos Manuel Brito Leal Queiróz,Fernando Manuel Fernandes da Costa Santos,1.0.2,None,None
4,7542,2018-06-20,14:00:00.000,International - FIFA World Cup,2018,Portugal,Morocco,1,0,available,...,2021-06-13T16:17:31.694,2,Group Stage,Stadion Luzhniki,Mark Geiger,Fernando Manuel Fernandes da Costa Santos,Hervé Renard,1.0.2,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,7540,2018-06-19,20:00:00.000,International - FIFA World Cup,2018,Russia,Egypt,3,1,available,...,2021-06-13T16:17:31.694,2,Group Stage,Saint-Petersburg Stadium,Enrique Cáceres,Stanislav Cherchesov,Héctor Raúl Cúper,1.0.2,None,None
60,8652,2018-07-07,20:00:00.000,International - FIFA World Cup,2018,Russia,Croatia,2,2,available,...,2021-06-13T16:17:31.694,5,Quarter-finals,\tOlimpiyskiy Stadion Fisht,Sandro Ricci,Stanislav Cherchesov,Zlatko Dalić,1.0.2,None,None
61,7563,2018-06-26,16:00:00.000,International - FIFA World Cup,2018,Denmark,France,0,0,available,...,2021-06-13T16:17:31.694,3,Group Stage,Stadion Luzhniki,Sandro Ricci,Åge Fridtjof Hareide,Didier Deschamps,1.0.2,None,None
62,7556,2018-06-24,17:00:00.000,International - FIFA World Cup,2018,Japan,Senegal,2,2,available,...,2021-06-13T16:17:31.694,2,Group Stage,\tEkaterinburg Arena,Gianluca Rocchi,Akira Nishino,Aliou Cissé,1.0.2,None,None


In [7]:
def añadir_match_id(ratings_df, matches_df):
    import pandas as pd

    # 1. Normalizar fechas
    ratings_df['date'] = pd.to_datetime(ratings_df['date'], dayfirst=True).dt.date
    matches_df['match_date'] = pd.to_datetime(matches_df['match_date']).dt.date

    # 2. Limpiar y estandarizar nombres
    def limpiar_equipo(equipo):
        return equipo.strip().lower().replace('á', 'a').replace('é', 'e').replace('í', 'i')\
                                     .replace('ó', 'o').replace('ú', 'u').replace('’', "'")

    ratings_df['team_clean'] = ratings_df['team'].apply(limpiar_equipo)
    matches_df['home_team_clean'] = matches_df['home_team'].apply(limpiar_equipo)
    matches_df['away_team_clean'] = matches_df['away_team'].apply(limpiar_equipo)

    # 3. Extraer nombres de equipos desde la columna "match" del ratings
    def extraer_equipos(match_str):
        try:
            partes = match_str.split(",")[0]
            local, visitante = partes.split(" - ")
            return limpiar_equipo(local), limpiar_equipo(visitante)
        except:
            return None, None

    ratings_df[['home_team_clean', 'away_team_clean']] = ratings_df['match'].apply(
        lambda x: pd.Series(extraer_equipos(x))
    )

    # 4. Crear clave de emparejamiento
    ratings_df['match_key'] = ratings_df['date'].astype(str) + "_" + \
                              ratings_df['home_team_clean'] + "_" + \
                              ratings_df['away_team_clean']

    matches_df['match_key'] = matches_df['match_date'].astype(str) + "_" + \
                              matches_df['home_team_clean'] + "_" + \
                              matches_df['away_team_clean']

    # 5. Merge final
    ratings_df = ratings_df.merge(
        matches_df[['match_key', 'match_id']],
        on='match_key',
        how='left'
    )

    return ratings_df


In [8]:


ratings_df_con_id = añadir_match_id(ratings, mundial_2018)


KeyboardInterrupt: 

In [ ]:
ratings_df_con_id["match_id"].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
6114    False
6115    False
6116    False
6117    False
6118    False
Name: match_id, Length: 6119, dtype: bool

In [ ]:
ratings_df_con_id.columns

Index(['competition', 'date', 'match', 'team', 'pos', 'pos_role', 'player',
       'rater', 'is_human', 'original_rating', 'goals', 'assists',
       'shots_ontarget', 'shots_offtarget', 'shotsblocked', 'chances2score',
       'drib_success', 'drib_unsuccess', 'keypasses', 'touches', 'passes_acc',
       'passes_inacc', 'crosses_acc', 'crosses_inacc', 'lballs_acc',
       'lballs_inacc', 'grduels_w', 'grduels_l', 'aerials_w', 'aerials_l',
       'poss_lost', 'fouls', 'wasfouled', 'clearances', 'stop_shots',
       'interceptions', 'tackles', 'dribbled_past', 'tballs_acc',
       'tballs_inacc', 'ycards', 'rcards', 'dangmistakes', 'countattack',
       'offsides', 'goals_ag_otb', 'goals_ag_itb', 'saves_itb', 'saves_otb',
       'saved_pen', 'missed_penalties', 'owngoals', 'degree_centrality',
       'betweenness_centrality', 'closeness_centrality', 'flow_centrality',
       'flow_success', 'betweenness2goals', 'win', 'lost', 'is_home_team',
       'minutesPlayed', 'game_duration', 'team

In [ ]:
# Lista original de columnas deseadas
columnas_deseadas = [
    'competition', 'date', 'match', 'team', 'pos', 'pos_role',
    'player', 'rater', 'is_human', 'original_rating', 'match_id', 'minutesPlayed'
]

# Filtrar solo las que están presentes en el DataFrame actual
columnas_presentes = [col for col in columnas_deseadas if col in ratings_df_con_id.columns]

# Crear el nuevo DataFrame filtrado
ratings_filtrado = ratings_df_con_id[columnas_presentes]


In [ ]:
import pandas as pd

def promediar_ratings_por_jugador(df_ratings):
    """
    Agrupa el dataset de ratings por jugador y partido, eliminando duplicados por rater
    y devolviendo la media de original_rating por jugador/partido.
    """
    columnas_clave = [
        'competition', 'date', 'match', 'team', 'pos',
        'pos_role', 'player', 'match_id', 'original_rating', 'minutesPlayed'
    ]

    # Verificación
    for col in columnas_clave:
        if col not in df_ratings.columns:
            raise ValueError(f"Falta la columna requerida: {col}")

    # Agrupamiento y promedio
    df_limpio = df_ratings[columnas_clave].groupby(
        ['competition', 'date', 'match', 'team', 'pos', 'pos_role', 'player', 'match_id', 'minutesPlayed'],
        as_index=False
    )['original_rating'].mean()

    return df_limpio


In [ ]:

# Aplicar la limpieza
df_ratings_limpio = promediar_ratings_por_jugador(ratings_filtrado)

In [ ]:
df_ratings_limpio = df_ratings_limpio.rename(columns={"minutesPlayed": "minutes_played"})


In [ ]:
df_ratings_limpio

,competition,date,match,team,pos,pos_role,player,match_id,minutes_played,original_rating
0,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DC,Ilya Kutepov,7525,90,6.455000
1,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DC,Sergei Ignashevich,7525,90,6.615000
2,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DL,Yuri Zhirkov,7525,90,6.492500
3,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DR,Mario Fernandes,7525,90,6.420000
4,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,FW,FW,Fedor Smolov,7525,70,5.817500
...,...,...,...,...,...,...,...,...,...,...
1701,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,MF,ML,Blaise Matuidi,8658,73,5.985000
1702,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,MF,MR,Kylian Mbappe,8658,90,5.935000
1703,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,Sub,Sub,Corentin Tolisso,8658,17,6.055000
1704,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,Sub,Sub,Nabil Fekir,8658,9,6.320000


In [ ]:
ratings_filtrado = df_ratings_limpio

In [ ]:
data = mundial_2018

In [ ]:
# Lista para guardar DataFrames de eventos
eventos_lista = []

# Descargar eventos de cada partido
for match_id in data['match_id']:

        eventos = sb.events(match_id=match_id)
        eventos['match_id'] = match_id  # Añadir identificador de partido
        eventos_lista.append(eventos)


# Unir todos los DataFrames en uno solo
eventos_data = pd.concat(eventos_lista, ignore_index=True)

# Guardar como archivo CSV
#df_eventos.to_csv('eventos_seriea_2016.csv', index=False)


In [ ]:
def perfil_jugadores(df_eventos):
    # Eliminar eventos sin jugador
    df = df_eventos.dropna(subset=['player_id', 'player']).copy()

    # Lista para guardar los resultados
    perfiles = []

    # Agrupar por jugador
    for player_id, grupo in df.groupby('player_id'):
        nombre = grupo['player'].iloc[-1]
        ultimo_equipo = grupo['team'].dropna().iloc[-1] if 'team' in grupo else None
        ultimo_team_id = grupo['team_id'].dropna().iloc[-1] if 'team_id' in grupo else None
        posiciones = grupo['position'].dropna().unique().tolist()
        partidos_jugados = grupo['match_id'].nunique()  # contar partidos únicos

        perfiles.append({
            'player_id': int(player_id),
            'player': nombre,
            'ultimo_equipo': ultimo_equipo,
            'ultimo_team_id': ultimo_team_id,
            'posiciones': posiciones,
            'partidos_jugados': partidos_jugados
        })

    return pd.DataFrame(perfiles)


perfiles_jugadores = perfil_jugadores(eventos_data)

In [ ]:
from rapidfuzz import process, fuzz

def hacer_matching_aproximado_fila_a_fila(df_ratings, df_statsbomb, umbral=85, alias_manual=None):
    """
    Empareja fila a fila el nombre del jugador en df_ratings con los nombres de df_statsbomb,
    filtrando por país (team) y manteniendo la estructura fila a fila.
    """
    jugadores_statsbomb = df_statsbomb[['player', 'ultimo_equipo']].drop_duplicates()
    resultados = []

    for _, row in df_ratings.iterrows():
        nombre_jugador = row['player']
        pais_jugador = row['team']

        # 1. Matching manual (prioritario)
        if alias_manual and nombre_jugador in alias_manual:
            mejor_match = alias_manual[nombre_jugador]
            score = 100
        else:
            # 2. Candidatos por país
            candidatos = jugadores_statsbomb[jugadores_statsbomb['ultimo_equipo'] == pais_jugador]
            nombres_candidatos = candidatos['player'].tolist()

            mejor_match, score, _ = process.extractOne(
                nombre_jugador,
                nombres_candidatos,
                scorer=fuzz.token_sort_ratio
            ) or (None, 0, None)

        resultados.append({
            'player': nombre_jugador,
            'team': pais_jugador,
            'player_statsbomb': mejor_match,
            'score': score,
            'revisar': score < umbral and nombre_jugador not in (alias_manual or {})
        })

    return pd.DataFrame(resultados)


In [ ]:
# Matching con nombres de StatsBomb
alias_manual = {
    "Angel Di Maria": "Ángel Fabián Di María Hernández"
}

df_matching = hacer_matching_aproximado_fila_a_fila(ratings_filtrado, perfiles_jugadores, umbral=85, alias_manual=alias_manual)

# Añadir el nombre del jugador corregido desde StatsBomb
ratings_matched = ratings_filtrado.merge(
    df_matching[['player', 'team', 'player_statsbomb']],
    on=['player', 'team'],
    how='left'
)

# Reemplazar columna player por el nombre oficial de StatsBomb
ratings_matched['player'] = ratings_matched['player_statsbomb']
ratings_matched.drop(columns=['player_statsbomb'], inplace=True)


In [ ]:
ratings_matched

,competition,date,match,team,pos,pos_role,player,match_id,minutes_played,original_rating
0,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DC,Ilya Kutepov,7525,90,6.455000
1,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DC,Ilya Kutepov,7525,90,6.455000
2,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DC,Ilya Kutepov,7525,90,6.455000
3,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DC,Ilya Kutepov,7525,90,6.455000
4,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DC,Sergei Ignashevich,7525,90,6.615000
...,...,...,...,...,...,...,...,...,...,...
6133,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,Sub,Sub,Nabil Fekir,8658,9,6.320000
6134,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,Sub,Sub,Nabil Fekir,8658,9,6.320000
6135,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,Sub,Sub,Steve Mandanda,8658,35,5.623333
6136,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,Sub,Sub,Steve Mandanda,8658,35,5.623333


In [ ]:
def limpiar_ratings_duplicados(df_ratings):
    """
    Elimina filas duplicadas del dataset de ratings promediado por jugador y partido.
    """
    df_limpio = df_ratings.drop_duplicates()
    return df_limpio


In [ ]:
df_limpio = limpiar_ratings_duplicados(ratings_matched)

In [ ]:
df_limpio

,competition,date,match,team,pos,pos_role,player,match_id,minutes_played,original_rating
0,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DC,Ilya Kutepov,7525,90,6.455000
4,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DC,Sergei Ignashevich,7525,90,6.615000
8,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DL,Yuri Zhirkov,7525,90,6.492500
11,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,DF,DR,Mário Figueira Fernandes,7525,90,6.420000
15,World Cup 2018,2018-06-14,"Russia - Saudi Arabia, 5 - 0",Russia,FW,FW,Fyodor Smolov,7525,70,5.817500
...,...,...,...,...,...,...,...,...,...,...
6112,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,MF,ML,Blaise Matuidi,8658,73,5.985000
6117,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,MF,MR,Kylian Mbappé Lottin,8658,90,5.935000
6124,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,Sub,Sub,Corentin Tolisso,8658,17,6.055000
6129,World Cup 2018,2018-07-15,"France - Croatia, 4 - 2",France,Sub,Sub,Nabil Fekir,8658,9,6.320000


In [ ]:
df_limpio.to_csv("datos/ratings/ratings_sep_mundial_2018.csv")